In [ ]:
import os
import json
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import seaborn as sns
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams["font.size"] = 30

In [ ]:
def load_data(filename):
    with open(filename, "r") as file:
        data = pd.DataFrame(json.load(file))
    data["total_dry_time"]        = data["dry_run_time"]          + data["creation_time"] + data["import_time"]
    data["total_simulation_time"] = data["simulation_times_mean"] + data["creation_time"] + data["import_time"]
    return data

### Scaling plots

Plot how the execution time scales w.r.t. the number of qubits, for various Qibo backends.
In order to generate the logs, you can use ``scripts/qibo.sh`` with a specific circuit and precision.

In [ ]:
def plot_scaling(quantity, precision, circuit="qft"):

    # Load data
    cpu = load_data("../qibo_cpu.dat")
    gpu = load_data("../qibo_gpu.dat")

    # Settings
    cpu_backends = ["numpy", "tensorflow", "qibotf", "qibojit"]
    gpu_backends = ["tensorflow", "qibotf", "qibojit"]

    # Prepare GPU data
    condition = (gpu["circuit"] == circuit) & (gpu["precision"] == precision)
    data = {f"{k} GPU": gpu[(gpu["library_options"] == f"backend={k}") & condition] for k in gpu_backends}

    # Prepare CPU data
    condition = (cpu["circuit"] == circuit) & (cpu["precision"] == precision)
    data.update({k: cpu[(cpu["library_options"] == f"backend={k}") & condition] for k in cpu_backends})

    # Plot data
    cpu_cp = sns.color_palette("Oranges", len(cpu_backends))
    gpu_cp = sns.color_palette("Purples", len(gpu_backends))
    plt.figure(figsize=(16, 9))
    plt.semilogy(data["numpy"]["nqubits"], data["numpy"][quantity], marker="s", markersize=10,
                 color=cpu_cp[0], linewidth=3.0, label="numpy")
    plt.semilogy(data["tensorflow"]["nqubits"], data["tensorflow"][quantity], marker="o", markersize=10,
                 color=cpu_cp[1], linewidth=3.0, label="tensorflow cpu")
    plt.semilogy(data["qibotf"]["nqubits"], data["qibotf"][quantity], marker="^", markersize=10,
                 color=cpu_cp[2], linewidth=3.0, label="qibotf cpu")
    plt.semilogy(data["qibojit"]["nqubits"], data["qibojit"][quantity],
                 color=cpu_cp[3], linewidth=3.0, label="qibojit cpu", marker="D", markersize=10)
    plt.semilogy(data["tensorflow GPU"]["nqubits"], data["tensorflow GPU"][quantity], marker="o", markersize=10,
                 color=gpu_cp[0], linewidth=3.0, label="tensorflow gpu")
    plt.semilogy(data["qibotf GPU"]["nqubits"], data["qibotf GPU"][quantity], marker="^", markersize=10,
                 color=gpu_cp[1], linewidth=3.0, label="qibotf gpu")
    plt.semilogy(data["qibojit GPU"]["nqubits"], data["qibojit GPU"][quantity],
                 color=gpu_cp[2], linewidth=3.0, label="qibojit gpu", marker="D", markersize=10)

    plt.title(f"{circuit}, {precision} precision")
    plt.xlabel("Number of qubits")
    if quantity == "total_dry_time":
        plt.ylabel("Total dry run time (sec)")
    elif quantity == "total_simulation_time":
        plt.ylabel("Total simulation time (sec)")
    plt.legend(fontsize="small")
    plt.savefig(f"qibo_scaling_{quantity}_{precision}.pdf", bbox_inches="tight")

In [ ]:
plot_scaling("total_simulation_time", "double")

In [ ]:
plot_scaling("total_dry_time", "double")

In [ ]:
plot_scaling("total_simulation_time", "single")

In [ ]:
plot_scaling("total_dry_time", "single")

### Bars plots

Plot how the execution time varies w.r.t. the backend, for a specific number of qubits (but for different circuits).
In order to generate the logs, you can use ``scripts/qibo.sh`` repeatedly with many different circuits.

In [ ]:
def plot_bars(precision, nqubits):

    # Load data
    cpu_data = load_data("../qibo_cpu.dat")
    gpu_data = load_data("../qibo_gpu.dat")
    gpu_data["library_options"] += " GPU"
    data = pd.concat([cpu_data, gpu_data])

    # Set plot params
    hatches = ['/', '\\', 'o', '-', 'x', '.', '*']
    width = 0.1
    quantities = ["import_time", "creation_time", "dry_run_time", "simulation_times_mean", "total_simulation_time"]
    circuit = ["qft", "variational", "supremacy", "qv", "bv"]
    backends = ["tensorflow", "qibotf", "qibojit", "tensorflow GPU", "qibotf GPU", "qibojit GPU"]
    cp = sns.color_palette("Purples", len(backends))
    ws = [-2*width, -width, 0, width, 2 * width, 3*width]
    base_condition = (data["nqubits"] == nqubits) & (data["precision"] == precision)

    
    # Plot the results
    xvalues = np.array(range(len(circuit)))
    plt.figure(figsize=(25, 9))
    plt.title(f"{nqubits} qubits - Total simulation time")
    for i, backend in enumerate(backends):
        condition = (data["library_options"] == f"backend={backend}") & base_condition
        heights = {q: np.array([float(data[condition & (data["circuit"] == circ)][q]) for circ in circuit])
                   for q in quantities}

        plt.bar(xvalues + ws[i], heights["total_simulation_time"], color=cp[i],
                align="center", width=width, alpha=1, label=backend, log=True,
                edgecolor='w', hatch=hatches[i])

    plt.xticks(xvalues, circuit)
    plt.ylabel("Total simulation time (sec)")
    plt.legend(fontsize="small", bbox_to_anchor=(1,1))
    plt.savefig(f"qibo_bars_{precision}_{nqubits}.pdf", bbox_inches="tight")


In [ ]:
plot_bars("double", 15)

In [ ]:
plot_bars("double", 28)

### Breakdown of total simulation time
In this bar plots a breakdown of the total simulation time is plotted for different circuits and number of qubits, while the Qibo backend is fixed.
In order to generate the logs, you can use ``scripts/qibo_circuits.sh``.

In [ ]:
def plot_breakdown(precision, backend="qibojit"):

    # Load data
    data = load_data("../qibo_circuits.dat")

    # Set plot params
    hatches = ['/', '\\', 'o', '-', 'x', '.', '*']
    width = 0.1
    quantities = ["import_time", "creation_time", "dry_run_time", "simulation_times_mean",
                  "total_simulation_time", "total_dry_time"]
    circuit = ["qft", "variational", "supremacy", "qv", "bv"]
    nqubits = [18, 24, 28]
    oranges = sns.color_palette("Oranges", len(circuit))
    greens  = sns.color_palette("Greens",  len(circuit))
    purples = sns.color_palette("Purples", len(circuit))
    ws = [-2*width, -width, 0, width, 2 * width, 3*width]
    base_condition = (data["precision"] == precision)


    # Plot the results
    xvalues = np.array(range(len(nqubits)))
    plt.figure(figsize=(25, 9))
    plt.title(f"{backend} - Breakdown of total run time")
    for i, circ in enumerate(circuit):
        condition = (data["library_options"] == f"backend={backend}") & (data["circuit"] == circ) & base_condition
        heights = {q: np.array([float(data[condition & (data["nqubits"] == n)][q]) for n in nqubits])
                   for q in quantities}

        norm = heights["total_dry_time"]
        plt.bar(xvalues + ws[i], heights["import_time"]/norm,
                color=oranges[i], align="center", width=width, alpha=1,
                label=circ, edgecolor='w', hatch=hatches[i])
        plt.bar(xvalues + ws[i], heights["creation_time"]/norm,
                color=greens[i], align="center", width=width, alpha=1, label=circ,
                edgecolor='w', hatch=hatches[i],
                bottom=heights["import_time"]/norm)
        plt.bar(xvalues + ws[i], heights["dry_run_time"]/norm,
                color=purples[i], align="center", width=width, alpha=1, label=circ,
                edgecolor='w', hatch=hatches[i],
                bottom=(heights["import_time"]+heights["creation_time"])/norm)

        
    plt.xticks(xvalues, nqubits)
    plt.xlabel("Number of qubits")
    plt.ylabel("Execution time (%)")
    legend_elements = [
        Patch(facecolor="w", edgecolor="k", hatch=hatches[i], label=circ) for i, circ in enumerate(circuit)
    ]
    legend_elements.append(Patch(color=oranges[2], label="Import time"))
    legend_elements.append(Patch(color=greens[2],  label="Creation time"))
    legend_elements.append(Patch(color=purples[2], label="Dry run time"))
    plt.legend(handles=legend_elements, bbox_to_anchor=(1,1))
    plt.savefig(f"{backend}_breakdown_{precision}_{nqubits}.pdf", bbox_inches="tight")


In [ ]:
plot_breakdown("double")